In [14]:
from collections import defaultdict
import numpy as np
from cyvcf2 import VCF
import pandas as pd
from functools import reduce

In [4]:
### Loading coding TR coordinates ###
coordinates = pd.read_csv("TR_intersect.txt", sep = "\t", header=None)

In [5]:
one_nr_per_sample_coding = defaultdict(int)
both_nr_per_sample_coding = defaultdict(int)

for i in range(1,23):
    vcf_addr = "/gymreklab-tscc/helia/ensembl/experiments/coding_regions/heterozygosity/coding_calls_" + str(i) + ".vcf.gz"
    vcf = VCF(vcf_addr)
    samples = vcf.samples
    for record in vcf:
        if int(record.POS) in list(coordinates[1]) and int(record.INFO['END']) in list(coordinates[2]):
            genotypes = record.genotypes
            for i in range(len(samples)):
                sample = samples[i]
                genotype = genotypes[i]
                if genotype[0] != -1 and (genotype[0] != 0 or genotype[1] != 0):
                    one_nr_per_sample_coding[sample] += 1

                if genotype[0] != -1 and (genotype[0] != 0 and genotype[1] != 0):
                    both_nr_per_sample_coding[sample] += 1

In [6]:
np.mean(list(one_nr_per_sample_coding.values()))

336.6299188007495

In [12]:
one_nr_per_sample_whole = defaultdict(int)
both_nr_per_sample_whole = defaultdict(int)


for i in range(1,23):
    print(i)
    gt_file = "/gymreklab-tscc/helia/ensembl/experiments/stats/info/gt_chr" + str(i) + ".txt"
    with open(gt_file) as f:
        for line in f:
            line = line.split()
            GT = line[3].split("/")
            if GT[0] == ".":
                continue
            if GT[0] != '0' and GT[1] != '0':
                both_nr_per_sample_whole[line[2]] += 1
                continue
            if GT[0] != '0' or GT[1] != '0':
                one_nr_per_sample_whole[line[2]] += 1

1
2


KeyboardInterrupt: 

In [15]:
### Merging all data ###
one_nr_per_sample_coding_df = pd.DataFrame(one_nr_per_sample_coding.items())
both_nr_per_sample_coding_df = pd.DataFrame(both_nr_per_sample_coding.items())

one_nr_per_sample_whole_df = pd.DataFrame(one_nr_per_sample_whole.items())
both_nr_per_sample_whole_df = pd.DataFrame(both_nr_per_sample_whole.items())

all_df_list = [one_nr_per_sample_coding_df, both_nr_per_sample_coding_df, 
               one_nr_per_sample_whole_df, both_nr_per_sample_whole_df]

all_df = reduce(lambda  left,right: pd.merge(left,right,on=[0],
                                            how='inner'), all_df_list)

ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat